# Guided Project: Profitable App Profiles for the App Store and Google Play Markets

## Open datasets as lists of lists of strings + explore the datasets

In [ ]:
def from_csv_url_to_list_of_lists_of_strings(url):
  from requests import get
  response = get(url)
  text = response.text.encode(encoding=response.encoding).decode(encoding='utf8')
  list_of_lines = text.split('\n')
  if list_of_lines[-1] == '':
    print(f'The empty last line was removed while processing {url}.')
    del list_of_lines[-1]
  from csv import reader
  return list(reader(list_of_lines))

url = 'https://dq-content.s3.amazonaws.com/350/googleplaystore.csv'
google_requests_get = from_csv_url_to_list_of_lists_of_strings(url)
url = 'https://dq-content.s3.amazonaws.com/350/AppleStore.csv'
apple_requests_get = from_csv_url_to_list_of_lists_of_strings(url)

The empty last line was removed while processing https://dq-content.s3.amazonaws.com/350/AppleStore.csv.


In [ ]:
android_apps = google_requests_get
ios_apps = apple_requests_get

Wczytujemy dane z plików AppleStore.csv oraz googleplaystore.csv za pomocą stworzonej funkcji from_csv_url_to_list_of_lists_of_strings.

Tworzymy funkcje do eksploracji, wypisania pierwszych wierszy w pliku oraz wypisania liczebności wszystkich kolumn i wierszy. Dodanie \n do printa powoduje przerwę w celu wyraźniejszego odczytu wyniku.

In [ ]:
def exploring_data(dataset, start, end, rows_and_columns=False):
  dataset_slice=dataset[start:end]
  for row in dataset_slice:
    print(row, '\n')

  if rows_and_columns:
    print('Liczba wierszy:', len(dataset))
    print('Liczba kolumn:', len(dataset[0]))

*Etap 3.* Przetestowanie funkcji na naszych zbiorach danych.

In [ ]:
exploring_data(android_apps, 0, 5, True)
print('\n')
exploring_data(ios_apps, 0, 5, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

Liczba wierszy: 10842
Liczba kolumn: 13


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count

## Deleting a flawed row

Usuwamy błędne rekordy ze zbioru danych. Tworzymy nową funkcję delete_wrong_rating_row. Bedzie sprawdzała, czy dany wiersz ma prawidłowo sformułowaną nazwe, tzn. czy wystąpi znak float.

In [ ]:
def delete_wrong_rating_row(n):
  rating = android_apps[n][2]
  if float(rating) > 5:
    del android_apps[n]
    print('Usunięto wiersz ' + str(n))
  else:
    print('Nic nie zostało usunięte')

In [ ]:
delete_wrong_rating_row(10473)

Nic nie zostało usunięte


## Discover duplicate apps

Musimy stworzyć listę, w której będziemy mieli pewność, że dana aplikacja nie jest powtórzona. Tworzymy puste krotki o nazwie duplicate_apps oraz unique_apps. Sprawdzamy czy w zbiorze danych app występuje kilkakrotne występowanie tej samej nazwy aplikacji, jeśli nie, to nazwa zostaje dodana do unique_apps. Jeśli występuje kilkakrotnie, nazwa zostaje dodana do duplicate_apps. Zliczamy obie pozycje.

In [ ]:
def duplicate_checkup(dataa, column=0, printuj=True):
  duplicate_apps = []
  unique_apps = []

  for app in dataa:
    name = app[column]
    if name in unique_apps:
      duplicate_apps.append(name)
    else:
      unique_apps.append(name)

  if printuj:
    print('Liczba zduplikowanych aplikacji:', len(duplicate_apps))
    print('Powinno być:', len(unique_apps))
  return len(unique_apps)

In [ ]:
duplicate_checkup(android_apps[1:], 0)
duplicate_checkup(ios_apps[1:], 1);

Liczba zduplikowanych aplikacji: 1181
Powinno być: 9659
Liczba zduplikowanych aplikacji: 2
Powinno być: 7195


Aplikacja Instagram jest zachowana w różnych rekordach. Można to sprawdzić poniższym kodem. Takim sprawdzeniem unikniemy mylnego usunięcia prawidłowych komórek.

In [ ]:
for app in android_apps:
  name=app[0]
  if name == "Instagram":
    print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


## Remove duplicate apps

Przechodzimy do usunięcia zduplikowanych aplikacji. Jeśli dana aplikacja pojawia się pierwszy raz, to dodajemy go do listy google_aps. Jeśli pojawi się po raz kolejny sprawdzamy jego reviews, a jeśli ta liczba jest większa od reviews poprzednika podmieniamy aplikację w liście google_apps.

In [ ]:
google_apps = {}
for app in android_apps[1:]:
  name = app[0]
  if name not in google_apps:
    google_apps[name] = app
  else:
    current_n_reviews = int(app[3])
    stored_n_reviews = int(google_apps[name][3])
    if current_n_reviews >= stored_n_reviews:
      google_apps[name] = app

In [ ]:
bez_duplikacji = list(google_apps.values())
duplicate_checkup(bez_duplikacji)

Liczba zduplikowanych aplikacji: 0
Powinno być: 9659


9659

Dla zbioru danych ios_apps:

In [ ]:
apps_ios = {}
for app in ios_apps[1:]:
  name = app[0]
  if name not in apps_ios:
    apps_ios[name] = app
  else:
    current_n_reviews = int(app[3])
    stored_n_reviews = int(apps_ios[name][3])
    if current_n_reviews >= stored_n_reviews:
      apps_ios[name] = app

In [ ]:
ios_bez_duplikacji = list(apps_ios.values())
duplicate_checkup(ios_bez_duplikacji)

Liczba zduplikowanych aplikacji: 0
Powinno być: 7197


7197

## Non-English characters

Następnym krokiem projektu jest wykonanie funkcji, która pozbędzie się znaków specjalnych z rekordu. Interesuje nas, czy wartość jest w typie string oraz czy jest napisana w języku angielskim, ponieważ inne języki mogą zakłócać odbiór danych. Ustawiamy odgórny warunek 127.

In [ ]:
def characters(string):
  for character in string:
    if ord(character) > 127:
      return False
  return True

Sprawdzamy funkcję na podanych przez projekt danych.

In [ ]:
print(characters("Instagram"))
print(characters("爱奇艺PPS -《欢乐颂2》电视剧热播"))
print(characters("'Docs To Go™ Free Office Suite"))
print(characters("Instachat 😜"))

True
False
False
False


Zgodnie z naszymi przypuszczeniami, funkcja zwróciła nam True jedynie w pierwszym przypadku, kiedy dostarczyliśmy dane w typie string napisaną w języku angielskim bez znaków specjalnych. Jednak nasza funkcja nie jest kompletna, ponieważ odrzuca aplikacje, która w rzeczywistości jest w języku angielskim, ale posiada emotikonę, tak jak to jest w przypadku Instachat 😜 oraz Go™. Edytując naszą funkcję, możemy temu zapobiec. Dodajemy warunek, żeby ilość znaków specjalnych nie pasujących do naszego kryterium nie przekraczała liczby 3.

In [ ]:
def characters(string):
  n_english=0

  for character in string:
    if ord(character) > 127:
      n_english += 1
    
  if n_english > 3:
      return False
  else:
    return True

Sprawdzamy teraz nasze dane.

In [ ]:
print(characters("Instagram"))
print(characters("爱奇艺PPS -《欢乐颂2》电视剧热播"))
print(characters("'Docs To Go™ Free Office Suite"))
print(characters("Instachat 😜"))

True
False
True
True


Nasza funkcja jest teraz kompletna.

Bazując na tej funkcji, możemy przefitrować oba zbiory danych w poszukiwaniu rekordów w języku angielskim. Sprawdza nazwę aplikacji w kolumnie 0 w kontekście znaków specjalnych i dodaje je do android_english dla pierwszego zbioru danych i do ios_english dla drugiego zbioru.

In [ ]:
android_english=[]
ios_english=[]

for app in bez_duplikacji:
    name=app[0]
    if characters(name):
      android_english.append(app)


for app in ios_bez_duplikacji:
    name=app[1]
    if characters(name):
      ios_english.append(app)

exploring_data(android_english, 0, 3, True)
print("\n")
exploring_data(ios_english, 0, 3, True)



['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'FAMILY', '3.9', '974', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

Liczba wierszy: 9614
Liczba kolumn: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '

Otrzymaliśmy wynik dla zbioru danych Android 9614, a dla zbioru danych iOS 6183.

Kolejną czynnością w celu oczyszczenia naszych danych jest podział zawartości naszych zbiorów na aplikacje darmowe i płatne. Wystarczy do tego celu przyrównanie ceny w kolumnie 4(dla iOS) oraz kolumny 7(dla Android) do liczby 0. Cena w zbiorze iOS jest jako float, dlatego musimy uwzględnić liczby po przecinku.

In [ ]:
ios_free = []
android_free = []

for app in ios_english:
    price = float(app[4])
    if price == 0.0:
        ios_free.append(app)
        
for app in android_english:
    price = app[7]
    if price == '0':
            android_free.append(app)

Sprawdzimy ilość aplikacji.

In [ ]:
print(len(android_free))
print(len(ios_free))

8864
3222


Jest 8864 darmowych aplikacji w zbiorze Android oraz 3222 darmowych aplikacji w zbiorze iOS.

## Most Popular Apps by Genre 

Naszym celem jest znalezienie aplikacji, które mają szansę na przyciągnięcie użytkowników, ponieważ wraz z dużym zainteresowaniem dostajemy większe wynagrodzenie. 

Strategią na sprawdzenie jakie są szanse na osiągnięcie sukcesu jest wypuszczenie wersji początkowej. Jeśli zostanie ona zostanie dobrze przyjęta, wypuszczamy jej wersję ostateczną. Jeśli po określonym czasie aplikacja będzie dobrze prosperować, można ją rozwijać i dodać wersję na iOS. W naszym projekcie szukamy aplikacji odnoszące sukcesy na obu rynkach.

Pierwszą rzeczą wartą przestudiowania jest sprawdzenie najczęściej występujących gatunków. Dla zbioru danych Android będzie to kolumna prime_genre a dla zbioru danych iOS będą to kolumny Genres i Category. Następnie można zbudować funkcję, która stworzy tzw. tabelę częstości, która zwróci nam wynik w postaci wartości procentowych. Bazując na tej funkcji można stworzyć drugą, ktorą wyświetli wartości procentowe w porządku malejącym.

Sprawdzamy kolumny w naszym zbiorze danych. Wbudowana funkcja sorted() nie działa poprawnie ze słownikami, dlatego musimy zamienić słownik w liste krotek, gdzie każda taka krotka będzie zawierała klucz słownika z odpowiadającą mu wartością. Wartość słownika jest na pierwszym miejscu, a jego klucz na drugim, to jest warunek dobrego działania tej funkcji.

Funkcja generuje tablicę częstości, która następnie jest przekształcana na listę krotek, a następnie stortuje ją w porządku malejącym.

In [ ]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = table[key] / total * 100
        table_percentages[key] = percentage
        
    return table_percentages

In [ ]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

Sprawdzamy teraz naszą funkcję dla zbioru danych Android oraz iOS. Zrobimy to najpierw dla drugiego zbioru, patrząc na kolumnę 11 w której znajdują się gatunki.

In [ ]:
display_table(ios_free, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Widzimy, że największą popularnością wśród darmowych aplikacji dla iOS cieszy się gatunek Games z przybliżonym wynikiem 58.16, co stanowi większą połowę wszystkich aplikacji. Na następnym miejscu jest gatunek Entertainment(7.9), a za nim Photo&Video(5). 

Sprawdźmy teraz wyniki dla zbioru danych Android dla kolumny 9(Gatunek) oraz kolumny 1(Kategoria).

In [ ]:
display_table(android_free, 1)
print('\n')
display_table(android_free, 9)

FAMILY : 18.94178700361011
GAME : 9.713447653429604
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.15072202166065
ENTERTAINMENT : 0.947653429602888
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 0.64

W tym przypadku największą popularnością cieszy się kategoria Family (19), a z gatunnków Tools (8,4). Skupmy się teraz na konkretnych gatunkach w zbiorze iOS.

In [ ]:
genres_ios = freq_table(ios_free, 11)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_free:
        genre_app = app[11]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


Widzimy, że największą średnią ma kategoria Social Networking z wartością 71548.3. Warto się przyjrzeć, które z aplikacji w tej kategorii cieszą się największą popularnością.

In [ ]:
for app in ios_free:
  if app[11] == "Social Networking":
    print(app[1], ":", app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

Przejdźmy teraz do danych dla Androida.

In [ ]:
categories_android = freq_table(android_free, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_free:
        category_app = app[1]
        if category_app == category:
            n_inistalls = app[5]
            n_inistalls = n_inistalls.replace(',', '')
            n_inistalls = n_inistalls.replace('+', '')
            total += float(n_inistalls)
            len_category += 1
    avg_n_inistalls = total / len_category
    print(category, ':', avg_n_inistalls)

ART_AND_DESIGN : 1986335.0877192982
FAMILY : 3698568.010720667
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
TOOLS : 10801391.298666667
DATING : 854028.8303030303
EDUCATION : 1841666.6666666667
ENTERTAINMENT : 11719761.904761905
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15594505.74912892
VIDEO_PLAYERS : 24727872.452830188
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
NEWS_AND_

Tutaj największy wynik przypisany jest ART_AND_DESIGN. Kolejna ciekawa opcja to również wcześniej wspomiane FAMILY. 

In [ ]:
for app in android_free:
    if app[1] == 'ART_AND_DESIGN' and (app[5] == '1,000,000+'):
        print(app[0], ':', app[5])

Infinite Painter : 1,000,000+
Garden Coloring Book : 1,000,000+
Text on Photo - Fonteee : 1,000,000+
Name Art Photo Editor - Focus n Filters : 1,000,000+


*Podsumowanie.* W trakcie projektu analizowany były zbiory danych zawierające informacje na temat aplikacji na rynku Android oraz iOS. Naszym głównym założeniem było znalezienie takiego gatunku, które w obydwu kategoriach mógłby osiągnąć wysokie wyniki. Największym zainteresowaniem cieszyłaby się aplikacja, która bazowałaby na social networking(w przypadku iOS) oraz na sztuce(w przypadku Androida).